In [ ]:

import json
import numpy as np
import matplotlib.pyplot as plt
try:
    import pyrealsense2 as rs
except ImportError:
    print("Unable to import pyrealsense2.")


class RealsenseSensor():
    """Class for interacting with a RealSense D400-series sensor.

    pyrealsense2 should be installed from source with the following
    commands:
    
    FIRST TRY PIP INSTALL!!!! If this does not work for you, then go ahead:

    >>> git clone https://github.com/IntelRealSense/librealsense
    >>> cd librealsense
    >>> mkdir build
    >>> cd build
    >>> cmake .. \
        -DBUILD_EXAMPLES=true \
        -DBUILD_WITH_OPENMP=false \
        -DHWM_OVER_XU=false \
        -DBUILD_PYTHON_BINDINGS=true \
        -DPYTHON_EXECUTABLE:FILEPATH=/path/to/your/python/library/ \
        -G Unix\ Makefiles
    >>> make -j4
    >>> sudo make install
    >>> export PYTHONPATH=$PYTHONPATH:/usr/local/lib
    """

    def __init__(self, sn):
        self._running = None,
        self.id = sn
        self._pipe = rs.pipeline()
        self._rs_cfg = rs.config()
        self._align = rs.align(rs.stream.color)
        self._intrinsics = {}
        ctx = rs.context()
        # prints out cams and S/N, which should be defined as id in config
        for d in ctx.devices:
            print("Cam {} connected.".format(d))
        
  

    def __del__(self):
        if self._running:
            self.stop()

    def _setup_pipe(self):
        self._rs_cfg.enable_device(self.id)
        self._rs_cfg.enable_stream(
            rs.stream.color,
            640,
            480,
            rs.format.bgr8,
            30
        )
        self._rs_cfg.enable_stream(
            rs.stream.depth,
            640,
            480,
            rs.format.z16,
            30
        )

    def _get_intrinsics(self):
        stream = self._profile.get_stream(rs.stream.color)
        obj = stream.as_video_stream_profile().get_intrinsics()
        self._intrinsics = obj

    def start(self):
        self._setup_pipe()
        self._profile = self._pipe.start(self._rs_cfg)
        self._get_intrinsics()
        self._set_depth_scale()
        for _ in range(5):
            self._pipe.wait_for_frames()
        self._running = True

    def stop(self):
        if not self._running:
            return False
        self._pipe.stop()
        self._running = False
        return True

    def _to_numpy(self, frame, dtype):
        arr = np.asanyarray(frame.get_data(), dtype=dtype)
        return arr

    def _read_image(self):
        
        frames = self._pipe.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()
        
        depth_image = self._to_numpy(depth_frame, np.float32)

        depth_image *= self._depth_scale
        color_im = self._to_numpy(color_frame, np.uint8)
        return color_im, depth_image
    
    
    def _set_depth_scale(self):
        sensor = self._profile.get_device().first_depth_sensor()
        self._depth_scale = sensor.get_depth_scale()
    
    def frames(self):
        color_im, depth_im = self._read_image()
        return color_im, depth_im



In [ ]:
import sampleClient
import cv2
sensor = RealsenseSensor("832112070584")
sensor.start()


In [ ]:
img_bgr, d = sensor.frames()
d.dtype

In [ ]:
plt.imshow(img_bgr)

In [ ]:
intrinsics = {
    "cx": sensor._intrinsics.ppx,
    "cy": sensor._intrinsics.ppy,
    "fx": sensor._intrinsics.fx,
    "fy": sensor._intrinsics.fy
}

In [ ]:
segmask = sampleClient.predictMask(d, **intrinsics)

In [ ]:
sampleClient.predictGQCNN_pj(img_bgr, d, **intrinsics)

In [ ]:
sampleClient.predictFCGQCNN_pj(img_bgr, d, segmask['masks'][0], **intrinsics)